In [6]:
from typing import List, Tuple, Dict, Any, Optional
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
DIR_QASM_FILES = Path(
    "..", "program_bank", "v003", "2024_10_18__10_19__qiskit"
)

SAMPLE_SIZE = 10

TMP_DIR = Path(
    "..", ".tmp"
)
TMP_DIR.mkdir(exist_ok=True)

In [3]:
def get_all_qasm_files(folder: str) -> List[str]:
    return list(Path(folder).rglob("*.qasm"))


all_qasm_files = get_all_qasm_files(DIR_QASM_FILES)


sample = np.random.choice(all_qasm_files, SAMPLE_SIZE, replace=False)
for s in sample:
    print(s)

../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_1193_ab5ccb_qc_pytket.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_563_2fca0d_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_303_4ad6f8_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_3001_72c272_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_3344_7fe6cc_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_1679_5bfd09_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_2703_9db25f_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_1409_552c35_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_505_d42562_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_3413_65401c_qc_pennylane.qasm


In [4]:
from bqskit import Circuit
from bqskit import compile as bcompile

# try to import all the circuits


def load_circuit(file: str) -> Optional[Circuit]:
    try:
        return Circuit.from_file(file)
    except Exception as e:
        print(f"Error loading {file}: {e}")
        return None


# get try to export all the circuits
def export_circuit(circuit: Circuit, file: str) -> None:
    try:
        circuit.save(file)
    except Exception as e:
        print(f"Error exporting {file}: {e}")
        return None


# load all the circuits / and export them in the current tmp director
circuits = [load_circuit(str(file)) for file in tqdm(sample)]
exported_files = [TMP_DIR / f"{i}.qasm" for i in range(SAMPLE_SIZE)]

for circuit, file in zip(circuits, exported_files):
    if circuit is not None:
        export_circuit(circuit, str(file))

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
from bqskit.ext import qiskit_to_bqskit
from bqskit.ext import bqskit_to_qiskit

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qreg = QuantumRegister(5)
creg = ClassicalRegister(5)
qc = QuantumCircuit(qreg, creg)
qc.h(0)
qc.cx(0, 1)
qc.measure_all(inplace=True)

print(qc.draw())

bqc = qiskit_to_bqskit(qc)

# export to qasm
bqc.save(str(TMP_DIR / "qiskit_to_bqskit.qasm"))


# qc_back = bqskit_to_qiskit(bqc)
# print(qc_back.draw())

        ┌───┐      ░ ┌─┐            
  q0_0: ┤ H ├──■───░─┤M├────────────
        └───┘┌─┴─┐ ░ └╥┘┌─┐         
  q0_1: ─────┤ X ├─░──╫─┤M├─────────
             └───┘ ░  ║ └╥┘┌─┐      
  q0_2: ───────────░──╫──╫─┤M├──────
                   ░  ║  ║ └╥┘┌─┐   
  q0_3: ───────────░──╫──╫──╫─┤M├───
                   ░  ║  ║  ║ └╥┘┌─┐
  q0_4: ───────────░──╫──╫──╫──╫─┤M├
                   ░  ║  ║  ║  ║ └╥┘
  c0: 5/══════════════╬══╬══╬══╬══╬═
                      ║  ║  ║  ║  ║ 
meas: 5/══════════════╩══╩══╩══╩══╩═
                      0  1  2  3  4 


## Try to fix pennylane order

In [12]:
from pennylane.tape import QuantumTape
import pennylane as qml
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qreg = QuantumRegister(5)
creg = ClassicalRegister(5)
qc = QuantumCircuit(qreg, creg)
qc.h(4)
qc.cx(3, 4)
qc.measure_all(inplace=True)
print(qc.draw())

# Convert Qiskit circuit to a simplified form
simplified_qiskit_circ = qc.decompose().decompose()
n_qubits = simplified_qiskit_circ.num_qubits

# add at least one empty operation on each qubit to avoid the order change
prefix_circ = QuantumCircuit(n_qubits)
for i in range(n_qubits):
    prefix_circ.id(i)
simplified_qiskit_circ = prefix_circ.compose(simplified_qiskit_circ)

# Define measurement and PennyLane device
measurements = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
circuit_fn = qml.from_qiskit(
    simplified_qiskit_circ, measurements=measurements)
dev = qml.device('default.qubit', wires=n_qubits)
qml_circuit = qml.QNode(circuit_fn, dev)

# Extract the QASM from the PennyLane QNode
with QuantumTape(shots=10) as tape:
    qml_circuit.construct([], {})
    qasm_str_pennylane = qml_circuit.tape.to_openqasm(wires=sorted(tape.wires))

print(qasm_str_pennylane)

# save to file
with open(TMP_DIR / "pennylane.qasm", "w") as f:
    f.write(qasm_str_pennylane)

                   ░ ┌─┐            
  q4_0: ───────────░─┤M├────────────
                   ░ └╥┘┌─┐         
  q4_1: ───────────░──╫─┤M├─────────
                   ░  ║ └╥┘┌─┐      
  q4_2: ───────────░──╫──╫─┤M├──────
                   ░  ║  ║ └╥┘┌─┐   
  q4_3: ───────■───░──╫──╫──╫─┤M├───
        ┌───┐┌─┴─┐ ░  ║  ║  ║ └╥┘┌─┐
  q4_4: ┤ H ├┤ X ├─░──╫──╫──╫──╫─┤M├
        └───┘└───┘ ░  ║  ║  ║  ║ └╥┘
  c3: 5/══════════════╬══╬══╬══╬══╬═
                      ║  ║  ║  ║  ║ 
meas: 5/══════════════╩══╩══╩══╩══╩═
                      0  1  2  3  4 
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
id q[0];
id q[1];
id q[2];
id q[3];
id q[4];
u3(1.5707963267948966,0.0,3.141592653589793) q[4];
cx q[3],q[4];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];



In [9]:
from mqt import qcec
import tempfile
from qiskit.qasm2 import dumps


def check_two_qasm_strings(str_qiskit, str_pytket):
    # create two temporary files
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_qiskit, tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_pytket:
        f_qiskit.write(str_qiskit)
        f_pytket.write(str_pytket)
        f_qiskit.flush()
        f_pytket.flush()
        # print(f_qiskit.name)
        # print(f_pytket.name)

        # verify the two files
        res = qcec.verify(f_qiskit.name, f_pytket.name)
        equivalence = str(res.equivalence)
        # print(f"Equivalence: {equivalence}")
        return equivalence

In [10]:
from qiskit.qasm2 import dumps
qiskit_str = dumps(qc)
print(qiskit_str)
# store to file
with open(TMP_DIR / "qiskit.qasm", "w") as f:
    f.write(qiskit_str)

OPENQASM 2.0;
include "qelib1.inc";
qreg q2[5];
creg c1[5];
creg meas[5];
h q2[4];
cx q2[3],q2[4];
barrier q2[0],q2[1],q2[2],q2[3],q2[4];
measure q2[0] -> meas[0];
measure q2[1] -> meas[1];
measure q2[2] -> meas[2];
measure q2[3] -> meas[3];
measure q2[4] -> meas[4];
